In [ ]:
import numpy as np
import pandas as pd

In [ ]:
credit =pd.read_csv('data\\tmdb_5000_credits.csv')
credit

In [ ]:
movies = pd.read_csv('data\\tmdb_5000_movies.csv')
movies.head(1)

In [ ]:
bolly= pd.read_csv('data\\IMDB-Movie-Dataset(2023-1951).csv')
bolly

In [ ]:
bolly.info()

In [ ]:
bolly.isnull().sum()

In [ ]:
bolly = bolly[['movie_id','movie_name','overview','genre','cast','director']]
bolly

In [ ]:
bolly = bolly.rename(columns={'movie_id': 'id', 'movie_name': 'title', 'genre': 'genres'})
bolly.head()

In [ ]:
bolly['overview']   = bolly['overview'].apply(lambda x: x.split() if isinstance(x, str) else x)
bolly['genres']   = bolly['genres'].apply(lambda x: x.split() if isinstance(x, str) else x)
bolly['cast']     = bolly['cast'].apply(lambda x: x.split() if isinstance(x, str) else x)
bolly['director'] = bolly['director'].apply(lambda x: x.split() if isinstance(x, str) else x)


In [ ]:
bolly

In [ ]:
#merge both movies and credits
movies = movies.merge(credit,on='title')

In [ ]:
movies.head(1)

In [ ]:
movies.isnull().sum()

In [ ]:
movies.info()

we extracting those column that is used to creating tag

In [ ]:
#genres ,# id, # keyword, # title, #overview, #cast, #crew
movies = movies[['id','title','overview','genres','keywords','cast','crew']]
movies

In [ ]:
#chechking the missing values in column
movies.isnull().sum()

In [ ]:
#drop the missing values of overview
movies.dropna(inplace=True)

In [ ]:
#checking duplicate values
movies.duplicated().sum()


In [ ]:
#checking genres how look like
movies.iloc[0].genres

In [ ]:
#genres is in dectiory convert in list
import ast

In [ ]:
def convert(text):
    L = []
    for i in ast.literal_eval(text):
        L.append(i['name']) 
    return L 


In [ ]:
movies.dropna(inplace=True)

In [ ]:
movies['genres'] = movies['genres'].apply(convert)
movies['genres']

In [ ]:
movies.head(5)

In [ ]:
#apply same thing on keyword
movies['keywords'] = movies['keywords'].apply(convert)
movies['keywords']


In [ ]:
movies.head()

In [ ]:
#extracting top three cast from cast 
def convert3(text):
    L = []
    counter = 0
    for i in ast.literal_eval(text):
        if counter < 3:
            L.append(i['name'])
            counter+=1
        else:
            break
    return L 


In [ ]:
movies['cast'] = movies['cast'].apply(convert3)
movies.head()

In [ ]:
movies['crew'][0]

In [ ]:
#extracting director form crew column
def fetch_director(text):
    L = []
    for i in ast.literal_eval(text):
        if i['job'] == 'Director':
            L.append(i['name'])
            break
    return L 


In [ ]:
movies['crew'] = movies['crew'].apply(fetch_director)

In [ ]:
movies.sample(5)

convertng overview from string to list to keep all column in list

In [ ]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())
movies.sample(5)
movies.info()

In [ ]:
#change the the crew name with director
movies = movies.rename(columns={'crew': 'director'})
movies.info()

In [ ]:
merged_df = pd.concat([bolly, movies], ignore_index=True)
merged_df.to_csv('data\\merged_output.csv', index=False)  # don't assign to 'film'
merged_df.head()  # ✅ This will show the top 5 rows

movies=merged_df
movies

In [ ]:
#removing space between words to no confusune to creating tags
def collapse(L):
    if isinstance(L, list):
        return [i.replace(" ", "") for i in L]
    return []  # or return L if you want to leave non-lists unchanged


In [ ]:
movies['cast']     = movies['cast'].apply(collapse)
movies['director'] = movies['director'].apply(collapse)
movies['genres']   = movies['genres'].apply(collapse)
movies['keywords'] = movies['keywords'].apply(collapse)


In [ ]:
movies.head()

In [ ]:
movies

In [ ]:
#creating tag
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['director']


In [ ]:
new_df= movies[['id','title','tags']]
new_df

In [ ]:
#converting tag from list to string 
new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))
new_df

In [ ]:
#converting into lower case
new_df['tags']=new_df['tags'].apply(lambda x:x.lower())

In [ ]:
new_df

In [ ]:
new_df['tags'][0]

converting simmiler words to root word using nltk 

In [ ]:
import nltk

In [ ]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [ ]:
def stem(text):
    y=[]
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [ ]:
new_df['tags']=new_df['tags'].apply(stem)

converting tags into multidimensional vector and finding nearest vector to find simillar movies

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
#chosing top 5000 word from list of all words
cv=CountVectorizer(max_features=5000,stop_words='english')

In [ ]:
vectors=cv.fit_transform(new_df['tags']).toarray()

In [ ]:
vectors

In [ ]:
vectors.shape

In [ ]:
#seeing the top 5000 words
cv.get_feature_names_out()

We Calculating the cosine disatance between all vecotr from each other 
lower cosine values means similar movies

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity=cosine_similarity(vectors)

In [ ]:
#similarity matrix that store distance from each vector
similarity

In [ ]:
#sort the list with index after sorting not loss the index
#sorting is on second
sorted(list(enumerate(similarity[0])),reverse=True,key=lambda x:x[1])


Now if we enter Movies 
then give five similar movies

In [ ]:
#feathening the movie index 
def recommend(movie):
    movie_index = new_df[new_df['title']==movie].index[0]
    distances= similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]
    for i in movies_list:
        print(new_df.iloc[i[0]].title)

In [ ]:
recommend('Jawan')

In [ ]:
import pickle

In [ ]:
pickle.dump(new_df,open('data\\movies.pkl','wb'))

In [ ]:
new_df.to_dict()

In [ ]:
pickle.dump(new_df.to_dict(),open('data\\movie_dict.pkl','wb'))

In [ ]:
pickle.dump(similarity, open('data\\similarity.pkl','wb'))